<a href="https://colab.research.google.com/github/LucianoFloriano/ManipulacaoBancoDados/blob/main/desafio_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
using Distributed
using Statistics, LinearAlgebra, Random

if nprocs() == 1
    addprocs(3)
end

@everywhere begin
    using Statistics, LinearAlgebra, Random

    function calcular_metricas(dados)
        n = length(dados)

        # Múltiplas métricas estatísticas
        media = mean(dados)
        mediana = median(dados)
        desvio_padrao = std(dados)

        # Cálculo seguro de assimetria e curtose
        if desvio_padrao > 0
            z_scores = (dados .- media) ./ desvio_padrao
            assimetria = mean(z_scores .^ 3)
            curtose = mean(z_scores .^ 4) - 3
        else
            assimetria = 0.0
            curtose = -3.0
        end

        # Autocorrelação (lag 1)
        if n > 1
            autocorr = cor(dados[1:end-1], dados[2:end])
        else
            autocorr = 0.0
        end

        # Quartis
        q25 = quantile(dados, 0.25)
        q75 = quantile(dados, 0.75)

        return (media, mediana, desvio_padrao, assimetria, curtose, autocorr, q25, q75)
    end
end

# Versão serial
function analisar_dados_serial(conjuntos_dados)
    resultados = []

    for (i, dados) in enumerate(conjuntos_dados)
        metricas = calcular_metricas(dados)
        push!(resultados, (i, metricas))
    end

    return resultados
end

# Versão paralela
function analisar_dados_paralelo(conjuntos_dados)
    resultados = pmap(calcular_metricas, conjuntos_dados)
    return [(i, resultados[i]) for i in 1:length(conjuntos_dados)]
end

# Gerar dados de teste CORRIGIDO
function gerar_dados_teste(n_conjuntos, tamanho_conjunto)
    conjuntos = []

    for i in 1:n_conjuntos
        if i % 4 == 1
            dados = randn(tamanho_conjunto) .+ 5.0  # Normal
        elseif i % 4 == 2
            dados = rand(tamanho_conjunto) .* 10.0  # Uniforme
        elseif i % 4 == 3
            # Exponencial usando transformação de uniforme
            dados = -log.(rand(tamanho_conjunto)) .* 2.0
        else
            dados = rand(tamanho_conjunto) .* 2.0 .- 1.0  # Uniforme [-1, 1]
        end
        push!(conjuntos, dados)
    end

    return conjuntos
end

# Teste
println("=== ANÁLISE DE MÚLTIPLOS CONJUNTOS DE DADOS ===")
n_conjuntos, tamanho_conjunto = 50, 10_000
conjuntos_dados = gerar_dados_teste(n_conjuntos, tamanho_conjunto)

println("Número de conjuntos: $n_conjuntos")
println("Tamanho de cada conjunto: $tamanho_conjunto")
println("Número de workers: $(nworkers())")

println("\nSerial:")
@time resultados_serial = analisar_dados_serial(conjuntos_dados)

println("Paralelo:")
@time resultados_paralelo = analisar_dados_paralelo(conjuntos_dados)

# Verificar resultados
println("\nComparação do primeiro conjunto:")
println("Serial - Média: $(round(resultados_serial[1][2][1], digits=4)), Desvio: $(round(resultados_serial[1][2][3], digits=4))")
println("Paralelo - Média: $(round(resultados_paralelo[1][2][1], digits=4)), Desvio: $(round(resultados_paralelo[1][2][3], digits=4))")

diferenças = [abs(resultados_serial[i][2][1] - resultados_paralelo[i][2][1]) for i in 1:n_conjuntos]
println("\nMaior diferença nas médias: $(maximum(diferenças))")

=== ANÁLISE DE MÚLTIPLOS CONJUNTOS DE DADOS ===
Número de conjuntos: 50
Tamanho de cada conjunto: 10000
Número de workers: 3

Serial:
  2.523241 seconds (2.53 M allocations: 157.752 MiB, 7.52% gc time, 91.85% compilation time)
Paralelo:
  7.369290 seconds (2.34 M allocations: 116.988 MiB, 0.41% gc time, 26.94% compilation time)

Comparação do primeiro conjunto:
Serial - Média: 4.9875, Desvio: 1.0009
Paralelo - Média: 4.9875, Desvio: 1.0009

Maior diferença nas médias: 0.0
